# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
#imports
import os
import time
import gradio as gr
import openai
from dotenv import load_dotenv
import re

load_dotenv(override=True)
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_KEY = os.getenv("GOOGLE_API_KEY")
GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL")

In [ ]:
# OpenAI / Gemini Client
def get_client(model_choice):
    """
    Return an OpenAI client configured for GPT or Gemini.
    """
    if model_choice == "OpenAI GPT-4":
        return openai.OpenAI(api_key=OPENAI_KEY)
    else:
        return openai.OpenAI(
            api_key=GOOGLE_KEY,
            base_url=GEMINI_BASE_URL,
        )

In [ ]:
# Fake Weather Tool
def get_weather(location):
    data = {
        "new york": {"temp": 72, "condition": "Partly Cloudy"},
        "london": {"temp": 59, "condition": "Rainy"},
        "tokyo": {"temp": 68, "condition": "Clear"},
    }
    info = data.get(location.lower(), {"temp": 75, "condition": "Sunny"})
    return f"Weather in {location}: {info['temp']}°F, {info['condition']}"


def maybe_use_tool(message):
    """
    Detect patterns like 'weather in <location>' (case-insensitive)
    and inject tool result.
    Supports multi-word locations, e.g. "New York" or "tokyo".
    """
    pattern = re.compile(r"weather\s+in\s+([A-Za-z\s]+)", re.IGNORECASE)
    match = pattern.search(message)

    if match:
        location = match.group(1).strip(" ?.,!").title()
        tool_result = get_weather(location)
        return f"{message}\n\n[Tool used: {tool_result}]"

    return message

In [ ]:
# prompt
SYSTEM_PROMPTS = {
    "General Assistant": "You are a helpful and polite AI assistant.",
    "Technical Expert": "You are an expert software engineer who writes clear, correct code.",
    "Creative Writer": "You are a creative storyteller who writes imaginative and emotional prose.",
    "Science Tutor": "You are a science teacher who explains ideas simply and clearly.",
}

In [ ]:
# ---------------------------------------------
# Build chat messages
# ---------------------------------------------
def build_messages(history, user_msg, persona):
    messages = [{"role": "system", "content": SYSTEM_PROMPTS[persona]}]
    for u, a in history:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": user_msg})
    return messages


# ---------------------------------------------
# Stream model output
# ---------------------------------------------
def stream_response(model_choice, messages):
    """
    Uses the same openai library to stream from GPT or Gemini.
    """
    client = get_client(model_choice)
    model = "gpt-4o-mini" if model_choice == "OpenAI GPT-4" else "gemini-2.5-flash"

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    reply = ""
    for chunk in stream:
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            reply += chunk.choices[0].delta.content
            yield reply
        time.sleep(0.01)

In [ ]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # 🤖 Unified GPT + Gemini Chat

    - 🔀 Choose model: **OpenAI GPT-4** or **Gemini 2.5 Flash**
    - 🧠 Pick the assistant persona (system prompt injection)
    - 🛠 Tool support: ask about weather

    **Weather tool tips:**
    - Ask: "What's the weather in London?"
    - Also works for: New York, Tokyo
    - If a city isn't known, it returns a default sunny forecast
    """
    )

    with gr.Row():
        model_choice = gr.Dropdown(
            ["OpenAI GPT-4", "Gemini 2.5 Flash"],
            value="OpenAI GPT-4",
            label="Model",
        )
        persona = gr.Dropdown(
            list(SYSTEM_PROMPTS.keys()),
            value="General Assistant",
            label="Persona",
        )

    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Ask about weather or coding...", label="Your message")
    gr.Markdown(
        "💡 Tip: You can ask about the weather in **London**, **New York**, or **Tokyo**. "
        "I'll call a local tool and include that info in my answer."
    )
    send = gr.Button("Send", variant="primary")
    clear = gr.Button("Clear")

    state = gr.State([])

    msg.submit(chat_fn, [msg, state, model_choice, persona], chatbot).then(
        lambda chat: chat, chatbot, state
    ).then(lambda: "", None, msg)

    send.click(chat_fn, [msg, state, model_choice, persona], chatbot).then(
        lambda chat: chat, chatbot, state
    ).then(lambda: "", None, msg)

    clear.click(lambda: ([], []), None, [chatbot, state], queue=False)

if __name__ == "__main__":
    demo.launch()
